Character level ground truth in an MNIST-like format. All digits have been resized to a fixed resolution of 32-by-32 pixels.
The original character bounding boxes are extended in the appropriate dimension to become square windows, so that resizing them to 32-by-32 pixels does not introduce aspect ratio distortions. 
Nevertheless this preprocessing introduces some distracting digits to the sides of the digit of interest. 
Loading the .mat files creates 2 variables:
X which is a 4-D matrix containing the images, and y which is a vector of class labels. To access the images, X(:,:,:,i) gives the i-th 32-by-32 RGB image, with class label y(i).

 #### pyplot.imshow
 Xarray-like or PIL image
The image data. Supported array shapes are:

(M, N): an image with scalar data. The values are mapped to colors using normalization and a colormap. See parameters norm, cmap, vmin, vmax.
(M, N, 3): an image with RGB values (0-1 float or 0-255 int).
(M, N, 4): an image with RGBA values (0-1 float or 0-255 int), i.e. including transparency.
The first two dimensions (M, N) define the rows and columns of the image.

Out-of-range RGB(A) values are clipped.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import scipy.io as sio
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator as datagen

image_ind = 900
train_data = sio.loadmat('drive/MyDrive/Colab Notebooks/train_32x32.mat') #train data type : dictionary
test_data=sio.loadmat('drive/MyDrive/Colab Notebooks/test_32x32.mat')

#extracting data
X_train, y_train = train_data['X'], train_data['y']
X_test, y_test = test_data['X'], test_data['y']
#(32, 32, 3, 26032)

classes = [0,1,2,3,4,5,6,7,8,9]
nb_classes = 10

X_train = np.transpose(X_train,(3,0,1,2))
X_test = np.transpose(X_test,(3,0,1,2))
# print(X_train.shape, X_test.shape)
# (73257, 32, 32, 3) (26032, 32, 32, 3)

#elements type : ndarray -> float32 & normalise
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

#convert y_train,y_test to one-hot vector
print(y_train[:3])
y_train = to_categorical(y_train) #(73257, 11)
y_test = to_categorical(y_test) #(26032, 11)
y_train = np.delete(y_train,(10), axis=1) #(73257, 10)
y_test = np.delete(y_test,(10), axis=1) #(26032, 10)
print(y_train[:3])

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
'''
i = np.random.randint(1, len(X_train))
print("Label %d is" % i, y_train[i])
plt.imshow(X_train[i])
'''
#keras.backend.clear_session()

aux_model = Sequential([
    Conv2D(32, (3, 3), padding='same', 
                           activation='relu',
                           input_shape=(32, 32, 3)),
    BatchNormalization(),
    Conv2D(32, (3, 3), padding='same', 
                        activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    
    Conv2D(64, (3, 3), padding='same', 
                           activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), padding='same',
                        activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    
    Conv2D(128, (3, 3), padding='same', 
                           activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), padding='same',
                        activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),    
    Dense(10,  activation='softmax')
])

lr_schedule = keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 10))
optimizer = Adam(lr=1e-4, amsgrad=True)
aux_model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                 metrics=['accuracy'])
# Fit model in order to determine best learning rate

history = aux_model.fit_generator(datagen.flow(X_train, y_train, batch_size=128),epochs=3, validation_data=(X_val, y_val),callbacks=[lr_schedule])
#callbacks=[lr_schedule]
test_loss, test_acc = model.evaluate(x=test_images, y=test_labels, verbose=0)

print('Test accuracy is: {:0.4f} \nTest loss is: {:0.4f}'.
      format(test_acc, test_loss))

[[1]
 [9]
 [2]]
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


AttributeError: ignored

* Transpose 전:

x_train: ndarray, (32,32,3,73257) : RGB image, three channels

y_train: ndarray, (73257,1) 원소도 모두 ndarray type

* Transpose 후:

(73257, 32, 32, 3) (26032, 32, 32, 3)

"""
plt.imshow(x_train[:,:,:,image_ind])
plt.show()

print(y_train[image_ind])
"""
#print(type(X_train[0,0,0,0]),type(X_train)) # <class 'numpy.float32'> <class 'numpy.ndarray'>

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 13, 13, 128)       73856     
_________________________________________________________________
flatten_5 (Flatten)          (None, 21632)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 11)                237963    
Total params: 313,611
Trainable params: 313,611
Non-trainable params: 0
_________________________________________________________________


Activation Function : ReLU

Loss Function : CrossEntropy

Optimizer : Adadelta

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
import scipy.io as sio
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator as datagen

image_ind = 900
train_data = sio.loadmat('drive/MyDrive/Colab Notebooks/train_32x32.mat') #train data type : dictionary
test_data=sio.loadmat('drive/MyDrive/Colab Notebooks/test_32x32.mat')

#extracting data
X_train, y_train = train_data['X'], train_data['y']
X_test, y_test = test_data['X'], test_data['y']
#(32, 32, 3, 26032)

classes = [0,1,2,3,4,5,6,7,8,9]
nb_classes = 10

X_train = np.transpose(X_train,(3,0,1,2))
X_test = np.transpose(X_test,(3,0,1,2))
# print(X_train.shape, X_test.shape)
# (73257, 32, 32, 3) (26032, 32, 32, 3)

#elements type : ndarray -> float32 & normalise
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

#convert y_train,y_test to one-hot vector
print(y_train[:3])
y_train = to_categorical(y_train) #(73257, 11)
y_test = to_categorical(y_test) #(26032, 11)
y_train = np.delete(y_train,(10), axis=1) #(73257, 10)
y_test = np.delete(y_test,(10), axis=1) #(26032, 10)
print(y_train[:3])

model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=X_train[0].shape)) 
model.add(MaxPooling2D(pool_size=(2,2))) 
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu')) 
model.add(Flatten()) 
model.add(Dense(len(y_train[0]), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adadelta',
metrics=['CategoricalAccuracy'])

model.fit(X_train, y_train, batch_size=128, epochs=2, validation_split = 0.1)

score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])
#pred=model.predict(X_test) print(np.shape(pred),pred[:3])

model.summary()

FileNotFoundError: ignored